In [1]:
# Importing libraries for data manipulation
import sqlite3
import pandas as pd

# Importing libraries for data visualization
import matplotlib.pyplot as plt
import numpy as np

# Importing libraries for data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Importing libraries for classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report

# Importing libraries for classification models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

from py_helpers.pipeline_helper import *
from py_helpers.db_helper import *

In [2]:
seed = 42
target = 'playoff'
data = './db/WNBA_filtered.db'
tables = ['Awards_Players', 'Coaches', 'Players', 'Players_Teams', 'Teams', 'Teams_Year']

In [3]:
awards_players_df = retrieve_data(data, tables[0])  
coaches_df = retrieve_data(data, tables[1])  
players_df = retrieve_data(data, tables[2])  
players_teams_df = retrieve_data(data, tables[3])  
teams_df = retrieve_data(data, tables[4])  
teams_year_df = retrieve_data(data, tables[5])
teams_year_df = teams_year_df.drop(columns=['cumulativePoints'])

In [4]:
# Join the DataFrames
df = teams_df.merge(players_teams_df, on=['year', 'tmID'], how='left') \
                   .merge(coaches_df, on=['year', 'tmID'], how='left') 
                   #.merge(teams_year_df, on=['year', 'tmID'], how='left')
                   #.merge(players_df, left_on='playerID', right_on='playerID', how='left') \

# Drop the columns that are not needed

list(df.columns)

['tmID',
 'year',
 'playoff',
 'principal_component_1',
 'principal_component_2',
 'playerID',
 'GP',
 'GS',
 'minutes',
 'points',
 'oRebounds',
 'dRebounds',
 'rebounds',
 'assists',
 'steals',
 'blocks',
 'turnovers',
 'PF',
 'fgRatio',
 'ftRatio',
 'threeRatio',
 'dq',
 'p_stint',
 'coachID',
 'won',
 'lost',
 'c_stint']

In [5]:
# Data from the eleveth year
coaches_csv = '../season_11/coaches.csv'
players_teams_csv = '../season_11/players_teams.csv'
teams_csv = '../season_11/teams.csv'

coaches_df_11 = pd.read_csv(coaches_csv)
coaches_df_11['c_stint'] = coaches_df_11['stint']
coaches_df_11 = coaches_df_11.drop(columns=['stint'])

players_teams_df_11 = pd.read_csv(players_teams_csv)
players_teams_df_11['p_stint'] = players_teams_df_11['stint']
players_teams_df_11 = players_teams_df_11.drop(['stint'], axis=1)

teams_df_11 = pd.read_csv(teams_csv)

In [6]:
# Having coaches_df and coaches_df_11
# Remove coaches_df_11 columns that are not in coaches_df
# Coaches_df could have more columns than coaches_df_11
coaches_df_11_cols = [x for x in coaches_df_11.columns if x in coaches_df.columns]
coaches_df_11 = coaches_df_11[coaches_df_11_cols]

# Having players_teams_df and players_teams_df_11
# Remove players_teams_df_11 columns that are not in players_teams_df
# players_teams_df could have more columns than players_teams_df_11
players_teams_df_11_cols = [x for x in players_teams_df_11.columns if x in players_teams_df.columns]
players_teams_df_11 = players_teams_df_11[players_teams_df_11_cols]

# Having df_teams and teams_df
# Remove df_teams columns that are not in teams_df
# teams_df could have more columns than df_teams
teams_df_11_cols = [x for x in teams_df_11.columns if x in teams_df.columns]
teams_df_11 = teams_df_11[teams_df_11_cols]

In [7]:
# Merge the DataFrames of the eleventh year
df_11 = teams_df_11.merge(players_teams_df_11, on=['year', 'tmID'], how='left') \
                     .merge(coaches_df_11, on=['year', 'tmID'], how='left')
df_11.columns

Index(['year', 'tmID', 'playerID', 'p_stint', 'coachID', 'c_stint'], dtype='object')

In [8]:
df_11.head()

,year,tmID,playerID,p_stint,coachID,c_stint
0,11,ATL,balesal01w,0,meadoma99w,0
1,11,ATL,castriz01w,0,meadoma99w,0
2,11,ATL,desouer01w,0,meadoma99w,0
3,11,ATL,hardili01w,0,meadoma99w,0
4,11,ATL,irvinsa01w,0,meadoma99w,0


In [9]:
team_names = df['tmID'].unique().tolist()
new_teams = df_11['tmID'].unique().tolist()
team_names = list(set(team_names + new_teams))
team_names.sort()
team_names

['ATL',
 'CHA',
 'CHI',
 'CLE',
 'CON',
 'DET',
 'HOU',
 'IND',
 'LAS',
 'MIA',
 'MIN',
 'NYL',
 'ORL',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'SEA',
 'TUL',
 'UTA',
 'WAS']

In [10]:
len(team_names)
encoder = LabelEncoder()
team_names = encoder.fit_transform(team_names)
print(team_names)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [11]:
print([x for x in df_11.columns if x in df.columns])
print(list(df_11.columns))

['year', 'tmID', 'playerID', 'p_stint', 'coachID', 'c_stint']
['year', 'tmID', 'playerID', 'p_stint', 'coachID', 'c_stint']


In [12]:
# Columns to solve
to_solve = [x for x in df.columns if x not in df_11.columns]
# Remove the target column
to_solve.remove(target)

print(list(to_solve))
print(len(to_solve))

['principal_component_1', 'principal_component_2', 'GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgRatio', 'ftRatio', 'threeRatio', 'dq', 'won', 'lost']
20


In [13]:
# Join df_11 to df and assing missing values to NaN
df_all =  df.merge(df_11, on=['year', 'tmID', 'playerID', 'p_stint', 'coachID', 'c_stint'], how='outer')
            
awards_coach_df = awards_players_df[ ['playerID', 'year', 'Coach_of_the_Year'] ]
awards_coach_df = awards_coach_df.rename(columns={'playerID': 'coachID'})

awards_players_df = awards_players_df.drop(columns=['Coach_of_the_Year'])

df_all = df_all.merge(teams_year_df, on=['year', 'tmID'], how='outer') \
                .merge(awards_players_df, on=['playerID', 'year'], how='left') \
                .merge(awards_coach_df, on=['coachID', 'year'], how='left') \
                .merge(players_df, left_on='playerID', right_on='playerID', how='left') \
       
df_all.head()

,tmID,year,playoff,principal_component_1,principal_component_2,playerID,GP,GS,minutes,points,...,height,weight,birthYear,pos_C,pos_C_F,pos_F,pos_F_C,pos_F_G,pos_G,pos_G_F
0,ATL,9,0.0,1.863011,1.406214,balesal01w,17.0,9.0,389.0,82.0,...,79.0,218.0,1985.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ATL,9,0.0,1.863011,1.406214,castriz01w,29.0,20.0,671.0,269.0,...,72.0,140.0,1982.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ATL,9,0.0,1.863011,1.406214,desouer01w,12.0,8.0,277.0,112.0,...,77.0,190.0,1982.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,ATL,9,0.0,1.863011,1.406214,haynikr01w,33.0,3.0,486.0,94.0,...,69.0,147.0,1983.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,ATL,9,0.0,1.863011,1.406214,lacyje01w,33.0,22.0,605.0,189.0,...,75.0,175.0,1983.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
# check if I have pairs tmID-year-playerID-coachID with duplicated values
df_all[['tmID', 'year', 'playerID', 'coachID']].duplicated().sum()

0

In [15]:
# Checking for missing values after the merge
# Check for missing values in playoff column
df_all[target].isnull().sum()

233

In [16]:
# Print the number of entries in year 11
print(df_all[df_all['year'] == 11].shape)

# Checking for missing values on playoff after the merge on year 11
print(df_all[df_all['year'] == 11][target].isnull().sum())

(175, 51)
175


In [17]:
# Distinct pairs of tmID-year when the year is 11
df_all[df_all['year'] == 11][['tmID', 'year']].drop_duplicates().shape

(12, 2)

In [18]:
# Print the entrnumber of entries that are not in year 11 but have missing values on playoff
print(df_all[(df_all['year'] != 11) & (df_all[target].isnull())].shape)

(58, 51)


In [19]:
df_old_shape = df_all.shape

# Remove entries that are not in year 11 but have missing values on playoff
df_all = df_all[~((df_all['year'] != 11) & (df_all['playoff'].isna()))]

df_updated_shape = df_all.shape

# Print the number of entries in each set and the number of entries removed
print(df_old_shape)
print(df_updated_shape)
print(df_old_shape[0] - df_updated_shape[0])

(2378, 51)
(2320, 51)
58


In [20]:
columns_with_missing_values = df_all.columns[df_all.isna().any()].tolist()
# Using loc to modify the DataFrame
# Fill the missing values with 0 except for the target column
columns_with_missing_values.remove(target)
df_all.loc[:, columns_with_missing_values] = df_all[columns_with_missing_values].fillna(0)

columns_with_missing_values = df_all.columns[df_all.isna().any()].tolist()
# Checking for missing values after the merge
# Check for missing values in each column
for column in columns_with_missing_values:
    print(column, df_all[column].isna().sum())

# Expecting to see no missing values

playoff 175


In [21]:
# List the columns that are not boolean or numeric
non_numeric_columns = df_all.select_dtypes(exclude=['bool', 'number']).columns.tolist() 
print(non_numeric_columns)

# Remove the non-numeric columns except for the tmID
non_numeric_columns.remove('tmID')
df_all = df_all.drop(columns=non_numeric_columns)


# Insert into the database
table_name = 'merged'
schema = schema_builder(df_all)
schema = f"Create Table {table_name} ({schema})"
insert_dataframe(df_all, table_name, data, schema)

['tmID', 'playerID', 'coachID']


In [22]:

join_coaches_df = coaches_df.merge(coaches_df_11, on=['year', 'tmID', 'coachID', 'c_stint'], how='outer')

join_coaches_df = join_coaches_df.sort_values(by=['year', 'coachID']) 

temp_df = join_coaches_df.copy()
temp_df['year'] = temp_df['year'] + 1

# find the duplicated columns in temp_df and join_coaches_df
duplicated_columns = [x for x in temp_df.columns if x in join_coaches_df.columns]
# remove year and coachID
duplicated_columns.remove('year')
duplicated_columns.remove('coachID')
print(duplicated_columns)

join_coaches_df = join_coaches_df.merge(temp_df, on=['year', 'coachID'], how='left')

for column in duplicated_columns:
    if column == 'tmID':
        continue
    if column == 'c_stint':
        continue
    join_coaches_df.loc[~join_coaches_df[column+'_y'].isna(), column+'_x'] = join_coaches_df[column+'_y']

for column in duplicated_columns:
    join_coaches_df = join_coaches_df.drop(columns=[column+'_y'])
    join_coaches_df = join_coaches_df.rename(columns={column+'_x': column})

join_coaches_df.tail()

['tmID', 'won', 'lost', 'c_stint']


,coachID,year,tmID,won,lost,c_stint
171,meadoma99w,11,ATL,18.0,16.0,0
172,reevech99w,11,MIN,NaN,NaN,0
173,richano99w,11,TUL,NaN,NaN,0
174,thibami99w,11,CON,16.0,18.0,0
175,whisejo99w,11,NYL,9.0,12.0,0


In [23]:
players_teams_df_11
players_teams_df

join_players_teams_df = players_teams_df.merge(players_teams_df_11, on=['year', 'tmID', 'playerID', 'p_stint'], how='outer')
# Sort by year and playerID
join_players_teams_df = join_players_teams_df.sort_values(by=['year', 'playerID'])

temp_df = join_players_teams_df.copy()
temp_df['year'] = temp_df['year'] + 1

duplicated_columns = [x for x in temp_df.columns if x in join_players_teams_df.columns]
duplicated_columns.remove('year')
duplicated_columns.remove('playerID')
print(duplicated_columns)

join_players_teams_df = join_players_teams_df.merge(temp_df, on=['year', 'playerID'], how='left')

for column in duplicated_columns:
    if column == 'tmID':
        continue
    if column == 'p_stint':
        continue
    join_players_teams_df.loc[~join_players_teams_df[column+'_y'].isna(), column+'_x'] = join_players_teams_df[column+'_y']

for column in duplicated_columns:
    join_players_teams_df = join_players_teams_df.drop(columns=[column+'_y'])
    join_players_teams_df = join_players_teams_df.rename(columns={column+'_x': column})

join_players_teams_df.tail()

['tmID', 'GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgRatio', 'ftRatio', 'threeRatio', 'dq', 'p_stint']


,playerID,year,tmID,GP,GS,minutes,points,oRebounds,dRebounds,rebounds,assists,steals,blocks,turnovers,PF,fgRatio,ftRatio,threeRatio,dq,p_stint
2066,wrighta01w,11,SEA,33.0,33.0,1072.0,404.0,44.0,73.0,117.0,129.0,50.0,10.0,85.0,119.0,0.46,0.91,0.27,5.0,0
2067,youngso01w,11,SAS,33.0,33.0,1113.0,600.0,69.0,146.0,215.0,54.0,44.0,16.0,62.0,67.0,0.45,0.77,0.31,1.0,0
2068,youngta01w,11,CHI,11.0,0.0,73.0,30.0,6.0,8.0,14.0,5.0,6.0,1.0,9.0,9.0,0.33,0.65,0.20,0.0,0
2069,youngta01w,11,CHI,10.0,6.0,129.0,50.0,10.0,20.0,30.0,7.0,7.0,4.0,11.0,18.0,0.35,0.89,0.25,0.0,0
2070,zellosh01w,11,IND,34.0,4.0,802.0,406.0,25.0,81.0,106.0,62.0,27.0,17.0,77.0,111.0,0.40,0.86,0.31,3.0,0


In [24]:
teams_df_11
teams_df

join_teams_df = teams_df.merge(teams_df_11, on=['year', 'tmID'], how='outer')

join_teams_df = join_teams_df.sort_values(by=['year', 'tmID'])

temp_df = join_teams_df.copy()
temp_df['year'] = temp_df['year'] + 1

duplicated_columns = [x for x in temp_df.columns if x in join_teams_df.columns]
duplicated_columns.remove('year')
duplicated_columns.remove('tmID')
duplicated_columns.remove(target)
print(duplicated_columns)

join_teams_df = join_teams_df.merge(temp_df, on=['year', 'tmID'], how='left')

for column in duplicated_columns:        
    join_teams_df.loc[~join_teams_df[column+'_y'].isna(), column+'_x'] = join_teams_df[column+'_y']

duplicated_columns.append(target)
for column in duplicated_columns:
    join_teams_df = join_teams_df.drop(columns=[column+'_y'])
    join_teams_df = join_teams_df.rename(columns={column+'_x': column})


join_teams_df.tail()

['principal_component_1', 'principal_component_2']


,tmID,year,playoff,principal_component_1,principal_component_2
149,PHO,11,NaN,5.311335,-0.122703
150,SAS,11,NaN,1.817702,0.310203
151,SEA,11,NaN,1.022263,-0.103796
152,TUL,11,NaN,NaN,NaN
153,WAS,11,NaN,1.631433,0.226761


In [25]:
df_shift = join_teams_df.merge(join_players_teams_df, on=['year', 'tmID'], how='left') \
                        .merge(join_coaches_df, on=['year', 'tmID'], how='left')

df_shift = df_shift.sort_values(by=['year', 'tmID', 'playerID', 'coachID'])

df_shift['year'] = df_shift['year'] - 1
df_shift = df_shift.merge(teams_year_df, on=['year', 'tmID'], how='left')
df_shift['year'] = df_shift['year'] + 1


df_shift.tail() 

,tmID,year,playoff,principal_component_1,principal_component_2,playerID,GP,GS,minutes,points,...,threeRatio,dq,p_stint,coachID,won,lost,c_stint,totalPoints,totalMinutes,yearlyWins
2367,WAS,11,NaN,1.631433,0.226761,langhcr01w,34.0,22.0,1013.0,408.0,...,0.00,0.0,0,laceytr99w,NaN,NaN,0,2585.0,6873.0,16.0
2368,WAS,11,NaN,1.631433,0.226761,milleke01w,34.0,25.0,675.0,150.0,...,0.33,0.0,0,laceytr99w,NaN,NaN,0,2585.0,6873.0,16.0
2369,WAS,11,NaN,1.631433,0.226761,phillta02w,NaN,NaN,NaN,NaN,...,NaN,NaN,0,laceytr99w,NaN,NaN,0,2585.0,6873.0,16.0
2370,WAS,11,NaN,1.631433,0.226761,thomaja01w,NaN,NaN,NaN,NaN,...,NaN,NaN,0,laceytr99w,NaN,NaN,0,2585.0,6873.0,16.0
2371,WAS,11,NaN,1.631433,0.226761,walkede01w,34.0,30.0,821.0,291.0,...,0.00,1.0,0,laceytr99w,NaN,NaN,0,2585.0,6873.0,16.0


In [26]:
df_shift['year'] = df_shift['year'] - 1

df_shift = df_shift.merge(awards_players_df, on=['playerID', 'year'], how='left') \
                .merge(awards_coach_df, on=['coachID', 'year'], how='left') \
                .merge(players_df, left_on='playerID', right_on='playerID', how='left')

df_shift['year'] = df_shift['year'] + 1


df_shift.tail()

,tmID,year,playoff,principal_component_1,principal_component_2,playerID,GP,GS,minutes,points,...,height,weight,birthYear,pos_C,pos_C_F,pos_F,pos_F_C,pos_F_G,pos_G,pos_G_F
2367,WAS,11,NaN,1.631433,0.226761,langhcr01w,34.0,22.0,1013.0,408.0,...,74.0,184,1986.0,0,0,0,1,0,0,0
2368,WAS,11,NaN,1.631433,0.226761,milleke01w,34.0,25.0,675.0,150.0,...,70.0,140,1978.0,0,0,0,0,0,1,0
2369,WAS,11,NaN,1.631433,0.226761,phillta02w,NaN,NaN,NaN,NaN,...,78.0,198,1989.0,1,0,0,0,0,0,0
2370,WAS,11,NaN,1.631433,0.226761,thomaja01w,NaN,NaN,NaN,NaN,...,69.0,145,1989.0,0,0,0,0,0,1,0
2371,WAS,11,NaN,1.631433,0.226761,walkede01w,34.0,30.0,821.0,291.0,...,75.0,168,1977.0,0,0,1,0,0,0,0


In [27]:
# Columns that shift have and df_all does not have
# And vice versa
print([x for x in df_shift.columns if x not in df_all.columns])
print([x for x in df_all.columns if x not in df_shift.columns])

['playerID', 'coachID']
[]


In [28]:
# Checking for missing values after the merge
# Check for missing values in playoff column
df_shift[target].isnull().sum()

177

In [29]:
# Columns with missing values
columns_with_missing_values = df_shift.columns[df_shift.isna().any()].tolist()
for column in columns_with_missing_values:
    print(column, df_shift[column].isna().sum())

playoff 177
principal_component_1 32
principal_component_2 32
GP 64
GS 64
minutes 64
points 64
oRebounds 64
dRebounds 64
rebounds 64
assists 64
steals 64
blocks 64
turnovers 64
PF 64
fgRatio 64
ftRatio 64
threeRatio 64
dq 64
won 80
lost 80
totalPoints 253
totalMinutes 253
yearlyWins 253
All_Star_Game_Most_Valuable_Player 2305
Defensive_Player_of_the_Year 2305
Kim_Perrot_Sportsmanship_Award 2305
Most_Improved_Player 2305
Most_Valuable_Player 2305
Rookie_of_the_Year 2305
Sixth_Woman_of_the_Year 2305
WNBA_All_Decade_Team_Honorable_Mention 2305
WNBA_All_Decade_Team 2305
WNBA_Finals_Most_Valuable_Player 2305
Coach_of_the_Year 2222


In [30]:
## The tmID, year where the missing values are located
#sample = df_shift[df_shift['playoff'].isna()][['tmID', 'year', 'playerID', 'coachID']]
## Remove duplicates from the sample
#sample = sample.drop_duplicates()
## Order by year and tmID
#sample = sample.sort_values(by=['year', 'tmID'])
#sample.head()

In [31]:
# Columns with missing values on year 11
columns_with_missing_values_11 = df_shift[df_shift['year'] == 11].columns[df_shift[df_shift['year'] == 11].isna().any()].tolist()
for column in columns_with_missing_values_11:
    print(column, df_shift[df_shift['year'] == 11][column].isna().sum())
    

playoff 177
principal_component_1 32
principal_component_2 32
GP 64
GS 64
minutes 64
points 64
oRebounds 64
dRebounds 64
rebounds 64
assists 64
steals 64
blocks 64
turnovers 64
PF 64
fgRatio 64
ftRatio 64
threeRatio 64
dq 64
won 80
lost 80
totalPoints 32
totalMinutes 32
yearlyWins 32
All_Star_Game_Most_Valuable_Player 170
Defensive_Player_of_the_Year 170
Kim_Perrot_Sportsmanship_Award 170
Most_Improved_Player 170
Most_Valuable_Player 170
Rookie_of_the_Year 170
Sixth_Woman_of_the_Year 170
WNBA_All_Decade_Team_Honorable_Mention 170
WNBA_All_Decade_Team 170
WNBA_Finals_Most_Valuable_Player 170
Coach_of_the_Year 164


In [32]:
# Distinct pairs of tmID-year when the year is 11
df_shift[df_shift['year'] == 11][['tmID', 'year']].drop_duplicates().shape

# Unique values of year tmmID in year 11
df_shift[df_shift['year'] == 11][['tmID', 'year']].drop_duplicates().values.tolist()

[['ATL', 11],
 ['CHI', 11],
 ['CON', 11],
 ['IND', 11],
 ['LAS', 11],
 ['MIN', 11],
 ['NYL', 11],
 ['PHO', 11],
 ['SAS', 11],
 ['SEA', 11],
 ['TUL', 11],
 ['WAS', 11]]

In [33]:
# Non numeric columns
non_numeric_columns = df_shift.select_dtypes(exclude=['bool', 'number']).columns.tolist()
non_numeric_columns.remove('tmID')
non_numeric_columns

# Remove the non-numeric columns except for the tmID
df_shift = df_shift.drop(columns=non_numeric_columns)

In [34]:
columns_with_missing_values = df_shift.columns[df_shift.isna().any()].tolist()
# Using loc to modify the DataFrame
# Fill the missing values with 0 except for the target column
columns_with_missing_values.remove(target)
df_shift.loc[:, columns_with_missing_values] = df_shift[columns_with_missing_values].fillna(0)

columns_with_missing_values = df_shift.columns[df_shift.isna().any()].tolist()
# Checking for missing values after the merge
# Check for missing values in each column
for column in columns_with_missing_values:
    print(column, df_shift[column].isna().sum())

# Expecting to see no playof on year 11

playoff 177


In [35]:
# Store the df_shift DataFrame into the database
table_name = 'merged_shift'
schema = schema_builder(df_shift)
schema = f"Create Table {table_name} ({schema})"
insert_dataframe(df_shift, table_name, data, schema)